<a href="https://colab.research.google.com/github/jsilbergDS/UniTox/blob/main/Ablations/Upload_Processor_DICT_Long_Prompt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPDF2
!pip install openai
!pip install tqdm
!pip install openai

In [ ]:
import requests
import io
import PyPDF2
from bs4 import BeautifulSoup
import pandas as pd

import json
import requests
import pandas as pd
from openai import OpenAI
import time
import numpy as np
from tqdm import tqdm
import re


In [ ]:
df_to_use = pd.read_csv("/content/drive/MyDrive/ClinicalTrials/dailymed_full_text_final.csv")
dictrank_df = pd.read_csv("Table S1-Table 1.csv")

In [ ]:
dictrank_df = pd.read_csv("Table S1-Table 1.csv")
dictrank_df["lower"] = dictrank_df["Generic/Proper Name(s)"].str.lower()
df_to_use["lower"] = df_to_use["generic_name"].str.lower()
dictrank_df = dictrank_df.merge(df_to_use,on="lower",how="inner")
dictrank_df = dictrank_df.groupby("lower").first().reset_index()

In [ ]:
dictrank_df

In [ ]:
#USE ME

drugs_list = []
drugs_merged_list = []
drugs_initial_responses = []
drugs_no_less_most_responses = []
drugs_yes_no_responses = []
drugs_descriptive_responses = []
answers_list = []
initial_prompts = []
no_less_most_prompts = []
yes_no_prompts = []
descriptive_prompts = []
moiety_list = []
fda_label_list = []
set_list = []

for index, row in tqdm(dictrank_df.iterrows(), total=dictrank_df.shape[0]):
    #url=url.replace("https://fdalabel.fda.gov:8443","https://nctr-crs.fda.gov")
    #if "nctr-crs.fda.gov" in url:
    #print(plain_text)
    #elif "accessdata" in url:
    #    r = requests.get(url)
    #    f = io.BytesIO(r.content)

    #    reader = PyPDF2.PdfReader(f)
    #    plain_text = "\n".join([reader.pages[i].extract_text().strip() for i in range(0,len(reader.pages))])
    #    plain_text=plain_text.strip()



    initial_prompt = """Provide a summary of all the parts of the drug label that discuss cardiotoxicity risks and cardiotoxicity reactions for this drug.
    In your summary of each sentence, clearly state whether the drug itself was associated with or caused the cardiotoxicity risk.
        In your definition of cardiotoxicity, consider (among other examples):
        Heart damage that is fatal or life-threatening, death, need heart transplantation, (acute) myocardial infarction, heart attack, heart or cardiac failure, congestive heart failure (CHF), cardiomyopathy,
        myocarditis, coronary artery disease, myocardial ischemia, left ventricular dysfunction, cardiogenic shock, coronary, artery insufficiency, valvular heart disease, endocarditis,
        Fatal or life-threatening arrhythmia, heart/cardiac arrest, cardiorespiratory arrest, torsade de pointes (TdP), AV block III, ventricular fibrillation, Brugada syndrome,
        Angina pectoris, increased angina, mitral valve regurgitation, heart valve thickening, cardio spasm, Ventricular tachycardia, long QT syndrome/QT interval
        prolongation, ventricular arrhythmias. Also consider Hypotension, hypertension, AV block I & II, atrial fibrillation, tachycardia, bradycardia, palptitations, and sinus node dysfunction."""


    no_less_most_prompt = """Given the above information about a drug, answer 'was this drug associated with No Cardiotoxicity, Less Cardiotoxicity, or Most Cardiotoxicity?'
        Now answer with just one word: No, Less, or Most"""
    yes_no_prompt = """Given the above information about a drug, answer 'was this drug associated with Cardiotoxicity?'
        Now, answer with just one word: Yes or No"""
    descriptive_prompt = """Given the above information about a drug, answer 'was this drug was associated with No Cardiotoxicity, Less Cardiotoxicity, or Most Cardiotoxicity?'
        In your definition of cardiotoxicity, consider (among other examples) the following to be Most Cardiotoxic:
        Heart damage that is fatal or life-threatening, death, need heart transplantation, (acute) myocardial infarction, heart attack, heart or cardiac failure, congestive heart failure (CHF), cardiomyopathy,
        myocarditis, coronary artery disease, myocardial ischemia, left ventricular dysfunction, cardiogenic shock, coronary, artery insufficiency, valvular heart disease, endocarditis,
        Fatal or life-threatening arrhythmia, heart/cardiac arrest, cardiorespiratory arrest, torsade de pointes (TdP), AV block III, ventricular fibrillation, Brugada syndrome,
        Angina pectoris, increased angina, mitral valve regurgitation, heart valve thickening, cardio spasm, Ventricular tachycardia, long QT syndrome/QT interval
        prolongation, ventricular arrhythmias.

        Consider the following (among other examples) to be less Cardiotoxic: Hypotension, hypertension, AV block I & II, atrial fibrillation, tachycardia, bradycardia, palptitations, and sinus node dysfunction
        Now, answer with one word just: 'No', 'Less' or 'Most'"""

    initial_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Read through the following drug label"},
        {"role": "user", "content":row["full_text"]},
        {"role": "user", "content":initial_prompt}
        ],
        stream=False, temperature=0,max_tokens=2000
        )
    no_less_most_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":no_less_most_prompt}
        ],
        stream=False, temperature=0

        )
    yes_no_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":yes_no_prompt}
        ],
        stream=False, temperature=0

        )
    descriptive_response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "assistant", "content":initial_response.choices[0].message.content.strip()},
        {"role": "user", "content":descriptive_prompt}
        ],
        stream=False, temperature=0

        )
    print(no_less_most_response.choices[0].message.content.strip()," ",yes_no_response.choices[0].message.content.strip())
    drugs_list.append(row["generic_name"])
    moiety_list.append(row["moiety"])
    drugs_initial_responses.append(initial_response.choices[0].message.content.strip())
    drugs_no_less_most_responses.append(no_less_most_response.choices[0].message.content.strip())
    drugs_yes_no_responses.append(yes_no_response.choices[0].message.content.strip())
    drugs_descriptive_responses.append(descriptive_response.choices[0].message.content.strip())
    initial_prompts.append(initial_prompt)
    no_less_most_prompts.append(no_less_most_prompt)
    yes_no_prompts.append(yes_no_prompt)
    descriptive_prompts.append(descriptive_prompt)
    set_list.append(row["set_id"])
    fda_label_list.append(row["urls"])
    if index % 100 == 0:
      temp_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"descriptive_determination":drugs_descriptive_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"descriptive_prompts":descriptive_prompts,"set_id":set_list,"urls":fda_label_list})
      temp_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DICT_long_prompt_temp_"+str(index)+".csv")
final_df = pd.DataFrame({"generic_name":drugs_list,"reasoning":drugs_initial_responses,"less_determination":drugs_no_less_most_responses,"binary_determination":drugs_yes_no_responses,"descriptive_determination":drugs_descriptive_responses,"initial_prompts":initial_prompts,"no_less_most_prompts":no_less_most_prompts,"yes_no_prompts":yes_no_prompts,"descriptive_prompts":descriptive_prompts,"set_id":set_list,"urls":fda_label_list})
final_df.to_csv("/content/drive/MyDrive/ClinicalTrials/Sunday_DICT_long_prompt_temp_final.csv")